#### ChatBot 을 통한 도메인에 맞는 양식으로 finetuning 해서 예측하기

In [1]:
import pandas as pd

In [2]:
train = pd.read_csv('train.csv')

In [3]:
train

,ID,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,대출등급
0,TRAIN_00000,12480000,36 months,6 years,RENT,72000000,18.90,15,부채 통합,0,0,0.0,0.0,0.0,C
1,TRAIN_00001,14400000,60 months,10+ years,MORTGAGE,130800000,22.33,21,주택 개선,0,373572,234060.0,0.0,0.0,B
2,TRAIN_00002,12000000,36 months,5 years,MORTGAGE,96000000,8.60,14,부채 통합,0,928644,151944.0,0.0,0.0,A
3,TRAIN_00003,14400000,36 months,8 years,MORTGAGE,132000000,15.09,15,부채 통합,0,325824,153108.0,0.0,0.0,C
4,TRAIN_00004,18000000,60 months,Unknown,RENT,71736000,25.39,19,주요 구매,0,228540,148956.0,0.0,0.0,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96289,TRAIN_96289,14400000,36 months,10+ years,MORTGAGE,210000000,9.33,33,신용 카드,0,974580,492168.0,0.0,0.0,C
96290,TRAIN_96290,28800000,60 months,10+ years,MORTGAGE,132000000,5.16,25,주택 개선,0,583728,855084.0,0.0,0.0,E
96291,TRAIN_96291,14400000,36 months,1 year,MORTGAGE,84000000,11.24,22,신용 카드,0,1489128,241236.0,0.0,0.0,A
96292,TRAIN_96292,15600000,36 months,5 years,MORTGAGE,66330000,17.30,21,부채 통합,2,1378368,818076.0,0.0,0.0,D


In [4]:
train.대출등급.nunique()

7

In [5]:
test = pd.read_csv('test.csv')
test

,ID,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수
0,TEST_00000,16800000,36 months,8 years,MORTGAGE,132000000,19.64,12,주택 개선,0,394692,146604.0,0.0,0.0
1,TEST_00001,8400000,36 months,5 years,RENT,89971200,15.84,25,부채 통합,0,0,0.0,0.0,0.0
2,TEST_00002,17280000,36 months,6 years,RENT,150000000,8.41,20,신용 카드,0,1786980,281820.0,0.0,0.0
3,TEST_00003,14400000,36 months,5 years,MORTGAGE,66000000,13.72,30,신용 카드,1,669024,281724.0,0.0,0.0
4,TEST_00004,27600000,36 months,5 years,RENT,55200000,30.50,12,신용 카드,0,1250052,614844.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64192,TEST_64192,30000000,36 months,3 years,MORTGAGE,78000000,22.08,27,부채 통합,2,1307532,763380.0,0.0,0.0
64193,TEST_64193,30000000,60 months,10+ years,MORTGAGE,109200000,12.06,26,부채 통합,0,960612,1245252.0,0.0,0.0
64194,TEST_64194,6120000,36 months,10+ years,RENT,39600000,28.80,33,부채 통합,0,131520,80880.0,0.0,0.0
64195,TEST_64195,11520000,36 months,10+ years,MORTGAGE,66000000,25.44,41,부채 통합,1,1339536,601872.0,0.0,0.0


### 결측치 확인

In [30]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96294 entries, 0 to 96293
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ID            96294 non-null  object 
 1   대출금액          96294 non-null  int64  
 2   대출기간          96294 non-null  object 
 3   근로기간          96294 non-null  object 
 4   주택소유상태        96294 non-null  object 
 5   연간소득          96294 non-null  int64  
 6   부채_대비_소득_비율   96294 non-null  float64
 7   총계좌수          96294 non-null  int64  
 8   대출목적          96294 non-null  object 
 9   최근_2년간_연체_횟수  96294 non-null  int64  
 10  총상환원금         96294 non-null  int64  
 11  총상환이자         96294 non-null  float64
 12  총연체금액         96294 non-null  float64
 13  연체계좌수         96294 non-null  float64
 14  대출등급          96294 non-null  object 
dtypes: float64(4), int64(5), object(6)
memory usage: 11.0+ MB


In [68]:
train

,ID,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,대출등급
0,TRAIN_00000,12480000,36 months,6 years,RENT,72000000,18.90,15,부채 통합,0,0,0.0,0.0,0.0,C
1,TRAIN_00001,14400000,60 months,10+ years,MORTGAGE,130800000,22.33,21,주택 개선,0,373572,234060.0,0.0,0.0,B
2,TRAIN_00002,12000000,36 months,5 years,MORTGAGE,96000000,8.60,14,부채 통합,0,928644,151944.0,0.0,0.0,A
3,TRAIN_00003,14400000,36 months,8 years,MORTGAGE,132000000,15.09,15,부채 통합,0,325824,153108.0,0.0,0.0,C
4,TRAIN_00004,18000000,60 months,Unknown,RENT,71736000,25.39,19,주요 구매,0,228540,148956.0,0.0,0.0,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96289,TRAIN_96289,14400000,36 months,10+ years,MORTGAGE,210000000,9.33,33,신용 카드,0,974580,492168.0,0.0,0.0,C
96290,TRAIN_96290,28800000,60 months,10+ years,MORTGAGE,132000000,5.16,25,주택 개선,0,583728,855084.0,0.0,0.0,E
96291,TRAIN_96291,14400000,36 months,1 year,MORTGAGE,84000000,11.24,22,신용 카드,0,1489128,241236.0,0.0,0.0,A
96292,TRAIN_96292,15600000,36 months,5 years,MORTGAGE,66330000,17.30,21,부채 통합,2,1378368,818076.0,0.0,0.0,D


In [8]:
train['대출기간'] = train['대출기간'].apply(lambda x: int(x.split(' ')[1]))

/tmp/ipykernel_1800/4247542419.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['대출기간'] = train['대출기간'].apply(lambda x: int(x.split(' ')[1]))


In [9]:
train['근로기간'] = train['근로기간'].apply(lambda x: x.replace('+', ' '))
train['근로기간'] = train['근로기간'].apply(lambda x: x.replace('<', ''))
train['근로기간'] = train['근로기간'].apply(lambda x: x.replace('>', ''))
train['근로기간'] = train['근로기간'].apply(lambda x: x.replace('Unknown', '0 years'))
train['근로기간'] = train['근로기간'].apply(lambda x: x.replace('years', 'year'))
train['근로기간'] = train['근로기간'].apply(lambda x: int(x.split('year')[0]))


/tmp/ipykernel_1800/293306862.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['근로기간'] = train['근로기간'].apply(lambda x: x.replace('+', ' '))
/tmp/ipykernel_1800/293306862.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['근로기간'] = train['근로기간'].apply(lambda x: x.replace('<', ''))
/tmp/ipykernel_1800/293306862.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [10]:
train['대출목적'] = train['대출목적'].apply(lambda x: x.replace('부채 통합', 'debt consolidation'))
train['대출목적'] = train['대출목적'].apply(lambda x: x.replace('주택 개선', 'housing improvement'))
train['대출목적'] = train['대출목적'].apply(lambda x: x.replace('주요 구매', 'major purchase'))
train['대출목적'] = train['대출목적'].apply(lambda x: x.replace('휴가', 'vacation'))
train['대출목적'] = train['대출목적'].apply(lambda x: x.replace('의료', 'medical treatment'))
train['대출목적'] = train['대출목적'].apply(lambda x: x.replace('자동차', 'car'))
train['대출목적'] = train['대출목적'].apply(lambda x: x.replace('신용 카드', 'credit card'))
train['대출목적'] = train['대출목적'].apply(lambda x: x.replace('소규모 사업', 'small business'))
train['대출목적'] = train['대출목적'].apply(lambda x: x.replace('기타', 'etc'))
train['대출목적'] = train['대출목적'].apply(lambda x: x.replace('이사', 'residential move'))
train['대출목적'] = train['대출목적'].apply(lambda x: x.replace('주택', 'house'))
train['대출목적'] = train['대출목적'].apply(lambda x: x.replace('재생 에너지', 'renewable energy'))



/tmp/ipykernel_1800/3457848075.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['대출목적'] = train['대출목적'].apply(lambda x: x.replace('부채 통합', 'debt consolidation'))
/tmp/ipykernel_1800/3457848075.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['대출목적'] = train['대출목적'].apply(lambda x: x.replace('주택 개선', 'housing improvement'))
/tmp/ipykernel_1800/3457848075.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

In [11]:
val['대출기간'] = val['대출기간'].apply(lambda x: int(x.split(' ')[1]))
val['근로기간'] = val['근로기간'].apply(lambda x: x.replace('+', ' '))
val['근로기간'] = val['근로기간'].apply(lambda x: x.replace('<', ''))
val['근로기간'] = val['근로기간'].apply(lambda x: x.replace('>', ''))
val['근로기간'] = val['근로기간'].apply(lambda x: x.replace('Unknown', '0 years'))
val['근로기간'] = val['근로기간'].apply(lambda x: x.replace('years', 'year'))
val['근로기간'] = val['근로기간'].apply(lambda x: int(x.split('year')[0]))
val['대출목적'] = val['대출목적'].apply(lambda x: x.replace('부채 통합', 'debt consolidation'))
val['대출목적'] = val['대출목적'].apply(lambda x: x.replace('주택 개선', 'housing improvement'))
val['대출목적'] = val['대출목적'].apply(lambda x: x.replace('주요 구매', 'major purchase'))
val['대출목적'] = val['대출목적'].apply(lambda x: x.replace('휴가', 'vacation'))
val['대출목적'] = val['대출목적'].apply(lambda x: x.replace('의료', 'medical treatment'))
val['대출목적'] = val['대출목적'].apply(lambda x: x.replace('자동차', 'car'))
val['대출목적'] = val['대출목적'].apply(lambda x: x.replace('신용 카드', 'credit card'))
val['대출목적'] = val['대출목적'].apply(lambda x: x.replace('소규모 사업', 'small business'))
val['대출목적'] = val['대출목적'].apply(lambda x: x.replace('기타', 'etc'))
val['대출목적'] = val['대출목적'].apply(lambda x: x.replace('이사', 'residential move'))
val['대출목적'] = val['대출목적'].apply(lambda x: x.replace('주택', 'house'))
val['대출목적'] = val['대출목적'].apply(lambda x: x.replace('재생 에너지', 'renewable energy'))




/tmp/ipykernel_1800/948104713.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val['대출기간'] = val['대출기간'].apply(lambda x: int(x.split(' ')[1]))
/tmp/ipykernel_1800/948104713.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val['근로기간'] = val['근로기간'].apply(lambda x: x.replace('+', ' '))
/tmp/ipykernel_1800/948104713.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

In [12]:
test['대출기간'] = test['대출기간'].apply(lambda x: int(x.split(' ')[1]))
test['근로기간'] = test['근로기간'].apply(lambda x: x.replace('+', ' '))
test['근로기간'] = test['근로기간'].apply(lambda x: x.replace('<', ''))
test['근로기간'] = test['근로기간'].apply(lambda x: x.replace('>', ''))
test['근로기간'] = test['근로기간'].apply(lambda x: x.replace('Unknown', '0 years'))
test['근로기간'] = test['근로기간'].apply(lambda x: x.replace('years', 'year'))
test['근로기간'] = test['근로기간'].apply(lambda x: int(x.split('year')[0]))
test['대출목적'] = test['대출목적'].apply(lambda x: x.replace('부채 통합', 'debt consolidation'))
test['대출목적'] = test['대출목적'].apply(lambda x: x.replace('주택 개선', 'housing improvement'))
test['대출목적'] = test['대출목적'].apply(lambda x: x.replace('주요 구매', 'major purchase'))
test['대출목적'] = test['대출목적'].apply(lambda x: x.replace('휴가', 'vacation'))
test['대출목적'] = test['대출목적'].apply(lambda x: x.replace('의료', 'medical treatment'))
test['대출목적'] = test['대출목적'].apply(lambda x: x.replace('자동차', 'car'))
test['대출목적'] = test['대출목적'].apply(lambda x: x.replace('신용 카드', 'credit card'))
test['대출목적'] = test['대출목적'].apply(lambda x: x.replace('소규모 사업', 'small business'))
test['대출목적'] = test['대출목적'].apply(lambda x: x.replace('기타', 'etc'))
test['대출목적'] = test['대출목적'].apply(lambda x: x.replace('이사', 'residential move'))
test['대출목적'] = test['대출목적'].apply(lambda x: x.replace('주택', 'house'))
test['대출목적'] = test['대출목적'].apply(lambda x: x.replace('재생 에너지', 'renewable energy'))




### Dataset 만들기

In [6]:
from datasets import Dataset, DatasetDict

In [7]:
train = train.iloc[ :-2000, : ]
val = train.iloc[-2000: , : ]

In [13]:
train_data = Dataset.from_pandas(train)
val_data = Dataset.from_pandas(val)


In [14]:
test_data = Dataset.from_pandas(test)

In [15]:
train_data

Dataset({
    features: ['ID', '대출금액', '대출기간', '근로기간', '주택소유상태', '연간소득', '부채_대비_소득_비율', '총계좌수', '대출목적', '최근_2년간_연체_횟수', '총상환원금', '총상환이자', '총연체금액', '연체계좌수', '대출등급'],
    num_rows: 94294
})

In [16]:
dataset = DatasetDict(
    {
        "train" : train_data,
        "val" : val_data,
        "test" : test_data
    }
)

In [17]:
dataset

DatasetDict({
    train: Dataset({
        features: ['ID', '대출금액', '대출기간', '근로기간', '주택소유상태', '연간소득', '부채_대비_소득_비율', '총계좌수', '대출목적', '최근_2년간_연체_횟수', '총상환원금', '총상환이자', '총연체금액', '연체계좌수', '대출등급'],
        num_rows: 94294
    })
    val: Dataset({
        features: ['ID', '대출금액', '대출기간', '근로기간', '주택소유상태', '연간소득', '부채_대비_소득_비율', '총계좌수', '대출목적', '최근_2년간_연체_횟수', '총상환원금', '총상환이자', '총연체금액', '연체계좌수', '대출등급'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['ID', '대출금액', '대출기간', '근로기간', '주택소유상태', '연간소득', '부채_대비_소득_비율', '총계좌수', '대출목적', '최근_2년간_연체_횟수', '총상환원금', '총상환이자', '총연체금액', '연체계좌수'],
        num_rows: 64197
    })
})

In [18]:
prompt_format = """ you are financial speciallist. And you will see ###condition of one person. and judge his/her Credit rating
response about ###instruction 

###Condition
he/she owe $ %d , take out a loan during %d months. 
He/she works for %d years. Status of home ownership is %s
His/her annual income was about %d. His/her total debt/income ratio was %f 
He/she have total %d accounts, and he/she take out a loan for %s.
He/she overdue of interest %d times over last 2 year.
His/Her total redemption principal $ %d. His/Her total interst pay $ %f 
His/Her total overdue payment $ %f. He/She have %f overdue account for total 


###instruction: with those conditions, Guess His Credit Rating in one of A,B,C,D,E,F,G 


###Credit Rating : %s 
"""

In [19]:




def gen_prompt(element):
    return DatasetDict({'input': prompt_format%(element['대출금액'], element['대출기간'], element['근로기간'], element['주택소유상태'], element['연간소득'], element['부채_대비_소득_비율'], element['총계좌수'], element['대출목적'], element['최근_2년간_연체_횟수'], element['총상환원금'],
element['총상환이자'], element['총연체금액'], element['연체계좌수'], element['대출등급'])})


dataset['train'] = dataset['train'].map(gen_prompt)

Map:   0%|          | 0/94294 [00:00<?, ? examples/s]

In [20]:
dataset['val'] = dataset['val'].map(gen_prompt)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [21]:
test_prompt_format = """you are financial speciallist. And you will see ###condition of one person. and judge his/her Credit rating
response about ###instruction 

###Condition
he/she owe $ %d , take out a loan during %d months. 
He/she works for %d years. Status of home ownership is %s
His/her annual income was about %d. His/her total debt/income ratio was %f 
He/she have total %d accounts, and he/she take out a loan for %s.
He/she overdue of interest %d times over last 2 year.
His/Her total redemption principal $ %d. His/Her total interst pay $ %f 
His/Her total overdue payment $ %f. He/She have %f overdue account for total 

###instruction: with those conditions, Guess His Credit Rating in one of A,B,C,D,E,F,G 


###Credit Rating : 

"""

In [22]:
def gen_prompt_test(element):
    return DatasetDict({'input': test_prompt_format%(element['대출금액'], element['대출기간'], element['근로기간'], element['주택소유상태'], element['연간소득'], element['부채_대비_소득_비율'], element['총계좌수'], element['대출목적'], element['최근_2년간_연체_횟수'], element['총상환원금'],
element['총상환이자'], element['총연체금액'], element['연체계좌수'])})

In [23]:
dataset['test'] = dataset['test'].map(gen_prompt_test)


Map:   0%|          | 0/64197 [00:00<?, ? examples/s]

In [24]:
ranking = train.대출등급.unique()
ranking

array(['C', 'B', 'A', 'D', 'F', 'G', 'E'], dtype=object)

In [25]:
int2label = {i: ranking[i] for i in range(len(ranking))}
label2int = {int2label[key]:key for key in int2label}

In [26]:
int2label, label2int

({0: 'C', 1: 'B', 2: 'A', 3: 'D', 4: 'F', 5: 'G', 6: 'E'},
 {'C': 0, 'B': 1, 'A': 2, 'D': 3, 'F': 4, 'G': 5, 'E': 6})

In [25]:
base_model = "beomi/open-llama-2-ko-7b"

### 토크나이저 , 모델 로드

In [26]:
from transformers import LlamaForCausalLM
from transformers import AutoTokenizer

In [27]:
base_model = "beomi/open-llama-2-ko-7b"


tokenizer = AutoTokenizer.from_pretrained(base_model, padding_side = 'right')




In [28]:
tokenizer.pad_token

'</s>'

In [32]:
from transformers import LlamaConfig

# llama2 모델 config 관련된 arg 정리 


configuration = LlamaConfig()

configuration

LlamaConfig {
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 2048,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "transformers_version": "4.36.2",
  "use_cache": true,
  "vocab_size": 32000
}

In [14]:
tokenizer.bos_token_id, tokenizer.eos_token_id, tokenizer.vocab_size


(1, 2, 46336)

In [96]:
configuration = LlamaConfig(**{
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 512, 
  "initializer_range": 0.02,
  "intermediate_size": 1376,
  "max_position_embeddings": 128,
  "model_type": "llama",
  "num_attention_heads": 4,
  "num_hidden_layers": 4,
  "pad_token_id": 2,
  "rms_norm_eps": 1e-06,
  "tie_word_embeddings": False,
  "transformers_version": "4.28.0",
  "use_cache": True,
  "vocab_size": 46336
})




# model check를 위한 코드이기 때문에 hidden_size 줄임. 
# attention head 줄임 
# vocan sisze의 경우는 tokenizer에서 학습된 토큰의 개수에 맞춰서 진행해준다. 

In [29]:
from transformers import LlamaForCausalLM


model = LlamaForCausalLM.from_pretrained(base_model)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [167]:
print(dataset['test']['input'][0])

you are financial speciallist. And you will see ###condition of one person. and judge his/her Credit rating
response about ###instruction 

###Condition
he/she owe $ 16800000 , take out a loan during 36 months. 
He/she works for 8 years. Status of home ownership is MORTGAGE
His/her annual income was about 132000000. His/her total debt/income ratio was 19.640000 
He/she have total 12 accounts, and he/she take out a loan for housing improvement.
He/she overdue of interest 0 times over last 2 year.
His/Her total redemption principal $ 394692. His/Her total interst pay $ 146604.000000 
His/Her total overdue payment $ 0.000000. He/She have 0.000000 overdue account for total 

###instruction: with those conditions, Guess His Credit Rating in one of A,B,C,D,E,F,G 


###Credit Rating : 




In [33]:
import torch 
torch.cuda.empty_cache()

In [155]:
##학습 전 결과 보기
tokenizer = AutoTokenizer.from_pretrained(base_model, padding_side = 'left')

outputs = tokenizer(
    dataset['test']['input'][0],
    truncation=True,
    max_length=2048,
    return_overflowing_tokens=True, # 자른 나머지를 버리는 것이 아닌 추후에 다시 사용하는 것을 의미한다. 
    return_length=True, # 길이를 같이 return 하게 된다.
    return_tensors = 'pt'
)
    


In [157]:
outputs

{'input_ids': tensor([[    1,   366,   526, 18161,   961,   455,   497,   391, 29889,  1126,
           366,   674,  1074,   835, 16122,   310,   697,  2022, 29889,   322,
         16833,   670, 29914,  2276, 24596,   277, 21700,    13,  5327,  1048,
           835,  2611,  4080, 29871,    13,    13,  2277, 29937, 25255,    13,
           354, 29914, 11360,   288,   705,   395, 29871, 29896, 29953, 29947,
         29900, 29900, 29900, 29900, 29900,  1919,  2125,   714,   263, 24806,
          2645, 29871, 29941, 29953,  7378, 29889, 29871,    13,  3868, 29914,
         11360,  1736,   363, 29871, 29947,  2440, 29889, 16034,   310,  3271,
         27428,   338,   341,  8476, 29954, 10461,    13, 29950,   275, 29914,
          2276, 17568, 17869,   471,  1048, 29871, 29896, 29941, 29906, 29900,
         29900, 29900, 29900, 29900, 29900, 29889,  3600, 29914,  2276,  3001,
          2553, 29873, 29914,   262,  2763, 11959,   471, 29871, 29896, 29929,
         29889, 29953, 29946, 29900, 2

In [158]:
model.to('cuda')

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(46336, 4096, padding_idx=2)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
 

In [159]:
outputs['length'][0] + 1

tensor(298)

In [163]:
from transformers import GenerationConfig


generate_config = GenerationConfig(
        pad_token_id = tokenizer.eos_token_id,
        do_sample=True,
        top_k=10,
        top_p = 0.9,
        num_return_sequences=1,
        repetition_penalty=1.1,
        max_new_tokens=outputs['length'][0] + 10,
        temperature = 0.8
    )






In [161]:

# Generate
generate_ids = model.generate(input_ids = outputs.input_ids.to('cuda'),
                              attention_mask = outputs.attention_mask.to('cuda')
                              , generation_config= generate_config)


result = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
print(result)

you are financial speciallist. And you will see ###condition of one person. and judge his/her Credit rating
response about ###instruction 

###Condition
he/she owe $ 16800000 , take out a loan during 36 months. 
He/she works for 8 years. Status of home ownership is MORTGAGE
His/her annual income was about 132000000. His/her total debt/income ratio was 19.640000 
He/she have total 12 accounts, and he/she take out a loan for housing improvement.
He/she overdue of interest 0 times over last 2 year.
His/Her total redemption principal $ 394692. His/Her total interst pay $ 146604.000000 
His/Her total overdue payment $ 0.000000. He/She have 0.000000 overdue account for total 

###instruction: with those conditions, Guess His Credit Rating in one of A,B,C,D,E,F,G 


###Credit Rating : 

#@시스템#사진#
#@이모티콘#
오케이~~^^❤️


In [162]:
tokenizer.padding_side

'left'

In [164]:
outputs = tokenizer(
    dataset['test']['input'][10],
    truncation=True,
    max_length=2048,
    return_overflowing_tokens=True, # 자른 나머지를 버리는 것이 아닌 추후에 다시 사용하는 것을 의미한다. 
    return_length=True, # 길이를 같이 return 하게 된다.
    return_tensors = 'pt'
)

In [165]:
generate_ids = model.generate(input_ids = outputs.input_ids.to('cuda'),
                              attention_mask = outputs.attention_mask.to('cuda')
                              , generation_config= generate_config)


result = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
print(result)

you are financial speciallist. And you will see ###condition of one person. and judge his/her Credit rating
response about ###instruction 

###Condition
he/she owe $ 16800000 , take out a loan during 60 months. 
He/she works for 10 years. Status of home ownership is MORTGAGE
His/her annual income was about 82680000. His/her total debt/income ratio was 21.960000 
He/she have total 27 accounts, and he/she take out a loan for debt consolidation.
He/she overdue of interest 0 times over last 2 year.
His/Her total redemption principal $ 199284. His/Her total interst pay $ 159924.000000 
His/Her total overdue payment $ 0.000000. He/She have 0.000000 overdue account for total 

###instruction: with those conditions, Guess His Credit Rating in one of A,B,C,D,E,F,G 


###Credit Rating : 

Guess the credit rating of A person

I think his credit rating is CCC+ 

###Condition:
A person owe $ 16800000 , take out a loan during 60 months. 
He/she works for 10 years. Status of home ownership is MORTGAG

In [166]:
print(result)

you are financial speciallist. And you will see ###condition of one person. and judge his/her Credit rating
response about ###instruction 

###Condition
he/she owe $ 16800000 , take out a loan during 60 months. 
He/she works for 10 years. Status of home ownership is MORTGAGE
His/her annual income was about 82680000. His/her total debt/income ratio was 21.960000 
He/she have total 27 accounts, and he/she take out a loan for debt consolidation.
He/she overdue of interest 0 times over last 2 year.
His/Her total redemption principal $ 199284. His/Her total interst pay $ 159924.000000 
His/Her total overdue payment $ 0.000000. He/She have 0.000000 overdue account for total 

###instruction: with those conditions, Guess His Credit Rating in one of A,B,C,D,E,F,G 


###Credit Rating : 

Guess the credit rating of A person

I think his credit rating is CCC+ 

###Condition:
A person owe $ 16800000 , take out a loan during 60 months. 
He/she works for 10 years. Status of home ownership is MORTGAG

### PEFT 를 통한 Instruction Tuning 하기

In [31]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device


from transformers import BitsAndBytesConfig
from transformers import LlamaForCausalLM
from transformers import AutoTokenizer

In [32]:
import gc
torch.cuda.empty_cache()
gc.collect()

599

In [33]:
use_4bit = True

In [34]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [66]:
bnb_4bit_compute_dtype = "bfloat16"


In [67]:
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
compute_dtype

torch.bfloat16

In [68]:
# Check GPU compatibility with bfloat16
if compute_dtype == torch.bfloat16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

Your GPU supports bfloat16: accelerate training with bf16=True


In [69]:
base_model = "beomi/open-llama-2-ko-7b"


tokenizer = AutoTokenizer.from_pretrained(base_model)

model = LlamaForCausalLM.from_pretrained(base_model, quantization_config=bnb_config, device_map = 'auto')


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [70]:
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training

In [71]:
list(TaskType)


model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)  # Explicitly specify!


In [72]:
list(TaskType)

[<TaskType.SEQ_CLS: 'SEQ_CLS'>,
 <TaskType.SEQ_2_SEQ_LM: 'SEQ_2_SEQ_LM'>,
 <TaskType.CAUSAL_LM: 'CAUSAL_LM'>,
 <TaskType.TOKEN_CLS: 'TOKEN_CLS'>,
 <TaskType.QUESTION_ANS: 'QUESTION_ANS'>,
 <TaskType.FEATURE_EXTRACTION: 'FEATURE_EXTRACTION'>]

In [73]:
import bitsandbytes as bnb

def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])


    # lm_head is often excluded.
    if 'lm_head' in lora_module_names:  # needed for 16-bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)


modules = find_all_linear_names(model)
modules

['k_proj', 'o_proj', 'gate_proj', 'down_proj', 'v_proj', 'up_proj', 'q_proj']

In [74]:
peft_config = LoraConfig(task_type=TaskType.CAUSAL_LM,
                        inference_mode=False, # 학습하는지  
                        r=8, # 작을 수록 trainable 한 파라미터의 개수가 낮아진ㄷ.ㅏ  
                        lora_alpha=16,  # scaling factor 
                        lora_dropout=0.1) # dropout 

model = get_peft_model(model, peft_config)


In [75]:
trainable, total = model.get_nb_trainable_parameters()
print(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")


Trainable: 4194304 | total: 6860050432 | Percentage: 0.0611%


In [76]:
model.print_trainable_parameters()

trainable params: 4,194,304 || all params: 6,860,050,432 || trainable%: 0.06114100824149743


In [77]:
def get_max_length(model):
    conf = model.config
    max_length = None
    for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            print(f"Found max length: {max_length}")
            break
    if not max_length:
        max_length = 1024
        print(f"Using default max length: {max_length}")
    return max_length


# Change the max length depending on hardware constraints.
max_length = get_max_length(model)
print(max_length)

Found max length: 2048
2048


In [78]:
tokenizer.pad_token = tokenizer.eos_token

In [79]:
dataset

DatasetDict({
    train: Dataset({
        features: ['ID', '대출금액', '대출기간', '근로기간', '주택소유상태', '연간소득', '부채_대비_소득_비율', '총계좌수', '대출목적', '최근_2년간_연체_횟수', '총상환원금', '총상환이자', '총연체금액', '연체계좌수', '대출등급', 'input'],
        num_rows: 94294
    })
    val: Dataset({
        features: ['ID', '대출금액', '대출기간', '근로기간', '주택소유상태', '연간소득', '부채_대비_소득_비율', '총계좌수', '대출목적', '최근_2년간_연체_횟수', '총상환원금', '총상환이자', '총연체금액', '연체계좌수', '대출등급', 'input'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['ID', '대출금액', '대출기간', '근로기간', '주택소유상태', '연간소득', '부채_대비_소득_비율', '총계좌수', '대출목적', '최근_2년간_연체_횟수', '총상환원금', '총상환이자', '총연체금액', '연체계좌수', 'input'],
        num_rows: 64197
    })
})

In [80]:
def tokenize(element):
    
    outputs = tokenizer(
        element['input'],
        truncation=True,
        max_length=2048
    )

    return {"input_ids": outputs["input_ids"]}


tokenized_datasets = dataset['train'].map(
    tokenize, batched=True, remove_columns=dataset['train'].column_names
)


Map:   0%|          | 0/94294 [00:00<?, ? examples/s]

In [81]:
tokenized_datasets

Dataset({
    features: ['input_ids'],
    num_rows: 94294
})

In [82]:
val_tokenized_datasets = dataset['val'].map(
    tokenize, batched=True, remove_columns=dataset['val'].column_names
)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [83]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)


In [84]:
model.config.use_cache = False


In [85]:
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    output_dir="peftllama0116",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    evaluation_strategy="steps",
    eval_steps=3000,
    logging_steps=100,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    weight_decay=0.1,
    warmup_steps=1_000,
    lr_scheduler_type="cosine",
    learning_rate=5e-4,
    fp16=True,
    push_to_hub=False,
    optim = "adamw_torch",
    save_strategy = "steps",
    save_steps = 1000,
    save_total_limit=2

)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets,
    eval_dataset=val_tokenized_datasets
)

In [86]:
trainer.train(resume_from_checkpoint='peftllama0116/checkpoint-17000')

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/vitruv/anaconda3/envs/mingyu/lib/python3.11/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [52]:
model.save_pretrained('chatbot0116_adapter__')

In [53]:
torch.save({}, 'chatbot0116_adapter__/pytorch_model.bin')

## inference

In [41]:
import os 

os.getcwd()

'/home/vitruv/ChatBot_mg'

In [29]:
from peft import PeftModel

In [30]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

NameError: name 'BitsAndBytesConfig' is not defined

In [43]:
base_model = "beomi/open-llama-2-ko-7b"


tokenizer = AutoTokenizer.from_pretrained('peftllama0116/checkpoint-17000', padding_side = 'left')

fir_model = LlamaForCausalLM.from_pretrained(base_model, quantization_config=bnb_config, device_map = 'auto')



Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [45]:
model = PeftModel.from_pretrained(fir_model, "peftllama0116/checkpoint-17000")


In [56]:
model.to('cuda')

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(46336, 4096, padding_idx=2)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear(in_features=4096, out_features=4096, bias=F

In [54]:
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(46336, 4096, padding_idx=2)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(in_features=4096, out_features=

In [55]:
dataset['test']['input'][0]

'you are financial speciallist. And you will see ###condition of one person. and judge his/her Credit rating\nresponse about ###instruction \n\n###Condition\nhe/she owe $ 16800000 , take out a loan during 36 months. \nHe/she works for 8 years. Status of home ownership is MORTGAGE\nHis/her annual income was about 132000000. His/her total debt/income ratio was 19.640000 \nHe/she have total 12 accounts, and he/she take out a loan for housing improvement.\nHe/she overdue of interest 0 times over last 2 year.\nHis/Her total redemption principal $ 394692. His/Her total interst pay $ 146604.000000 \nHis/Her total overdue payment $ 0.000000. He/She have 0.000000 overdue account for total \n\n###instruction: with those conditions, Guess His Credit Rating in one of A,B,C,D,E,F,G \n\n\n###Credit Rating : \n\n'

In [70]:
outputs = tokenizer(
        dataset['test']['input'][0],
        truncation = True, 
        max_length = 2048,
        return_tensors = 'pt',
        return_length = True
        )


In [65]:
from transformers import GenerationConfig


generate_config = GenerationConfig(
        pad_token_id = tokenizer.eos_token_id,
        do_sample=True,
        top_k=1,
        top_p = 0.9,
        num_return_sequences=1,
        repetition_penalty=1.1,
        max_new_tokens=100,
        temperature = 0.8
    )



In [72]:
import torch 

with torch.no_grad():
    generate_ids = model.generate(input_ids = outputs.input_ids.to('cuda'),
                                attention_mask = outputs.attention_mask.to('cuda')
                                , generation_config= generate_config)


    result = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    print(result)


you are financial speciallist. And you will see ###condition of one person. and judge his/her Credit rating
response about ###instruction 

###Condition
he/she owe $ 16800000 , take out a loan during 36 months. 
He/she works for 8 years. Status of home ownership is MORTGAGE
His/her annual income was about 132000000. His/her total debt/income ratio was 19.640000 
He/she have total 12 accounts, and he/she take out a loan for housing improvement.
He/she overdue of interest 0 times over last 2 year.
His/Her total redemption principal $ 394692. His/Her total interst pay $ 146604.000000 
His/Her total overdue payment $ 0.000000. He/She have 0.000000 overdue account for total 

###instruction: with those conditions, Guess His Credit Rating in one of A,B,C,D,E,F,G 


###Credit Rating : 

###Rating Description : 

###Total Debt : 

###Total Income : 

###Total Debt/Income Ratio : 

###Other Conditions : 

###Credit Rating : B 

###Rating Description : 

###Total Debt : 

###Total Income : 

###

In [73]:
outputs = tokenizer(
        dataset['test']['input'][10],
        truncation = True, 
        max_length = 2048,
        return_tensors = 'pt',
        return_length = True
        )


In [77]:
## if prompt engineering more !

In [79]:
print(dataset['test']['input'][10])

you are financial speciallist. And you will see ###condition of one person. and judge his/her Credit rating
response about ###instruction 

###Condition
he/she owe $ 16800000 , take out a loan during 60 months. 
He/she works for 10 years. Status of home ownership is MORTGAGE
His/her annual income was about 82680000. His/her total debt/income ratio was 21.960000 
He/she have total 27 accounts, and he/she take out a loan for debt consolidation.
He/she overdue of interest 0 times over last 2 year.
His/Her total redemption principal $ 199284. His/Her total interst pay $ 159924.000000 
His/Her total overdue payment $ 0.000000. He/She have 0.000000 overdue account for total 

###instruction: with those conditions, Guess His Credit Rating in one of A,B,C,D,E,F,G 


###Credit Rating : 




In [74]:
import torch 

with torch.no_grad():
    generate_ids = model.generate(input_ids = outputs.input_ids.to('cuda'),
                                attention_mask = outputs.attention_mask.to('cuda')
                                , generation_config= generate_config)


    result = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    print(result)


you are financial speciallist. And you will see ###condition of one person. and judge his/her Credit rating
response about ###instruction 

###Condition
he/she owe $ 16800000 , take out a loan during 60 months. 
He/she works for 10 years. Status of home ownership is MORTGAGE
His/her annual income was about 82680000. His/her total debt/income ratio was 21.960000 
He/she have total 27 accounts, and he/she take out a loan for debt consolidation.
He/she overdue of interest 0 times over last 2 year.
His/Her total redemption principal $ 199284. His/Her total interst pay $ 159924.000000 
His/Her total overdue payment $ 0.000000. He/She have 0.000000 overdue account for total 

###instruction: with those conditions, Guess His Credit Rating in one of A,B,C,D,E,F,G 


###Credit Rating : 

###Rating Description : 

###Total Debt : 

###Total Annual Interest Payment : 

###Overdue Payment : 

###Debt Consolidation : 

###Other Condition : 

###Credit Rating : B 

###Rating Description : 

###Total

In [80]:
tuned_prompt = """

you are financial speciallist. And you will see ###condition of one person. and judge his/her Credit rating
response about ###instruction 

###Condition
he/she owe $ 16800000 , take out a loan during 60 months. 
He/she works for 10 years. Status of home ownership is MORTGAGE
His/her annual income was about 82680000. His/her total debt/income ratio was 21.960000 
He/she have total 27 accounts, and he/she take out a loan for debt consolidation.
He/she overdue of interest 0 times over last 2 year.
His/Her total redemption principal $ 199284. His/Her total interst pay $ 159924.000000 
His/Her total overdue payment $ 0.000000. He/She have 0.000000 overdue account for total 

with those conditions, Guess His Credit Rating in one of A,B,C,D,E,F,G 

"""

In [81]:
outputs = tokenizer(
        tuned_prompt,
        truncation = True, 
        max_length = 2048,
        return_tensors = 'pt',
        return_length = True
        )


In [82]:
import torch 

with torch.no_grad():
    generate_ids = model.generate(input_ids = outputs.input_ids.to('cuda'),
                                attention_mask = outputs.attention_mask.to('cuda')
                                , generation_config= generate_config)


    result = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    print(result)




you are financial speciallist. And you will see ###condition of one person. and judge his/her Credit rating
response about ###instruction 

###Condition
he/she owe $ 16800000 , take out a loan during 60 months. 
He/she works for 10 years. Status of home ownership is MORTGAGE
His/her annual income was about 82680000. His/her total debt/income ratio was 21.960000 
He/she have total 27 accounts, and he/she take out a loan for debt consolidation.
He/she overdue of interest 0 times over last 2 year.
His/Her total redemption principal $ 199284. His/Her total interst pay $ 159924.000000 
His/Her total overdue payment $ 0.000000. He/She have 0.000000 overdue account for total 

with those conditions, Guess His Credit Rating in one of A,B,C,D,E,F,G 


###instruction: with those conditions, Guess His Credit Rating in one of A,B,C,D,E,F,G 


###Credit Rating : C 


###justification : overdue payment was zero 


###derivation : see attached file 


###recommendation : no need to take any action 

In [84]:
tuned_prompt2 = """ 
you are financial speciallist. And you will see ###condition of one person. and judge his/her Credit rating
response about ###instruction 

###Condition
he/she owe $ 7200000 , take out a loan during 36 months. 
He/she works for 1 years. Status of home ownership is MORTGAGE
His/her annual income was about 102000000. His/her total debt/income ratio was 19.910000 
He/she have total 34 accounts, and he/she take out a loan for debt consolidation.
He/she overdue of interest 1 times over last 2 year.
His/Her total redemption principal $ 550020. His/Her total interst pay $ 106968.000000 
His/Her total overdue payment $ 0.000000. He/She have 0.000000 overdue account for total 

###instruction: with those conditions, Guess His Credit Rating in one of A,B,C,D,E,F,G 


"""

In [85]:
outputs = tokenizer(
        tuned_prompt2,
        truncation = True, 
        max_length = 2048,
        return_tensors = 'pt',
        return_length = True
        )


In [86]:
import torch 

with torch.no_grad():
    generate_ids = model.generate(input_ids = outputs.input_ids.to('cuda'),
                                attention_mask = outputs.attention_mask.to('cuda')
                                , generation_config= generate_config)


    result = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    print(result)


 
you are financial speciallist. And you will see ###condition of one person. and judge his/her Credit rating
response about ###instruction 

###Condition
he/she owe $ 7200000 , take out a loan during 36 months. 
He/she works for 1 years. Status of home ownership is MORTGAGE
His/her annual income was about 102000000. His/her total debt/income ratio was 19.910000 
He/she have total 34 accounts, and he/she take out a loan for debt consolidation.
He/she overdue of interest 1 times over last 2 year.
His/Her total redemption principal $ 550020. His/Her total interst pay $ 106968.000000 
His/Her total overdue payment $ 0.000000. He/She have 0.000000 overdue account for total 

###instruction: with those conditions, Guess His Credit Rating in one of A,B,C,D,E,F,G 


###Credit Rating : A 


###Justification For The Credit Rating : overdue payment was not made within 30 days


###Debtors should not be given credit if they have been overdue payment more than once in last 2 year.


###If there wa

In [88]:
import torch 

with torch.no_grad():
    generate_ids = model.generate(input_ids = outputs.input_ids.to('cuda'),
                                attention_mask = outputs.attention_mask.to('cuda')
                                , generation_config= generate_config)


    result = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    print(result)


 
you are financial speciallist. And you will see ###condition of one person. and judge his/her Credit rating
response about ###instruction 

###Condition
he/she owe $ 7200000 , take out a loan during 36 months. 
He/she works for 1 years. Status of home ownership is MORTGAGE
His/her annual income was about 102000000. His/her total debt/income ratio was 19.910000 
He/she have total 34 accounts, and he/she take out a loan for debt consolidation.
He/she overdue of interest 1 times over last 2 year.
His/Her total redemption principal $ 550020. His/Her total interst pay $ 106968.000000 
His/Her total overdue payment $ 0.000000. He/She have 0.000000 overdue account for total 

###instruction: with those conditions, Guess His Credit Rating in one of A,B,C,D,E,F,G 


###Credit Rating : A 


###Justification For The Credit


In [104]:
print(result)

 
you are financial speciallist. And you will see ###condition of one person. and judge his/her Credit rating
response about ###instruction 

###Condition
he/she owe $ 7200000 , take out a loan during 36 months. 
He/she works for 1 years. Status of home ownership is MORTGAGE
His/her annual income was about 102000000. His/her total debt/income ratio was 19.910000 
He/she have total 34 accounts, and he/she take out a loan for debt consolidation.
He/she overdue of interest 1 times over last 2 year.
His/Her total redemption principal $ 550020. His/Her total interst pay $ 106968.000000 
His/Her total overdue payment $ 0.000000. He/She have 0.000000 overdue account for total 

###instruction: with those conditions, Guess His Credit Rating in one of A,B,C,D,E,F,G 


###Credit Rating : A 


###Justification For The Credit


In [109]:
import re




re.findall("###Credit Rating : ([A-Z]+)", result)

['A']

### Test Dataset 에 대하여 정답 만들기

In [35]:
base_model = "beomi/open-llama-2-ko-7b"


tokenizer = AutoTokenizer.from_pretrained(base_model)

first_model = LlamaForCausalLM.from_pretrained(base_model, quantization_config=bnb_config, device_map = 'auto')


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [36]:
model = PeftModel.from_pretrained(first_model, "chatbot0116_adapter__")


In [37]:
test_df = dataset['test'].to_pandas()

In [38]:
test_df['input'] = test_df['input'].apply(lambda x : x.replace('###instruction: ' , '').replace('###Credit Rating :', ''))

you are financial speciallist. And you will see ###condition of one person. and judge his/her Credit rating
response about ###instruction 

###Condition
he/she owe $ 16800000 , take out a loan during 36 months. 
He/she works for 8 years. Status of home ownership is MORTGAGE
His/her annual income was about 132000000. His/her total debt/income ratio was 19.640000 
He/she have total 12 accounts, and he/she take out a loan for housing improvement.
He/she overdue of interest 0 times over last 2 year.
His/Her total redemption principal $ 394692. His/Her total interst pay $ 146604.000000 
His/Her total overdue payment $ 0.000000. He/She have 0.000000 overdue account for total 

with those conditions, Guess His Credit Rating in one of A,B,C,D,E,F,G 


 




In [42]:
def chatbot(prompt):


    outputs = tokenizer(
        prompt,
        truncation = True, 
        max_length = 2048,
        return_tensors = 'pt',
        return_length = True
        )
    


    with torch.no_grad():
        generate_ids = model.generate(input_ids = outputs.input_ids.to('cuda'),
                                    attention_mask = outputs.attention_mask.to('cuda')
                                    , generation_config= generate_config)


    result = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    return result


In [43]:
from tqdm import tqdm

In [72]:
answer = [] 
for i in tqdm(test_df['input']):
    answer.append(chatbot(i))
    


100%|██████████| 64197/64197 [64:22:14<00:00,  3.61s/it]   


In [73]:
test_df['answer'] = answer 


test_df.to_csv('answer.csv', index = False)

In [74]:
print(test_df['input'][0])


##token 20 vs 100의 결과 

you are financial speciallist. And you will see ###condition of one person. and judge his/her Credit rating
response about ###instruction 

###Condition
he/she owe $ 16800000 , take out a loan during 36 months. 
He/she works for 8 years. Status of home ownership is MORTGAGE
His/her annual income was about 132000000. His/her total debt/income ratio was 19.640000 
He/she have total 12 accounts, and he/she take out a loan for housing improvement.
He/she overdue of interest 0 times over last 2 year.
His/Her total redemption principal $ 394692. His/Her total interst pay $ 146604.000000 
His/Her total overdue payment $ 0.000000. He/She have 0.000000 overdue account for total 

with those conditions, Guess His Credit Rating in one of A,B,C,D,E,F,G 


 




In [71]:
print(test_df['answer'][500])

you are financial speciallist. And you will see ###condition of one person. and judge his/her Credit rating
response about ###instruction 

###Condition
he/she owe $ 18000000 , take out a loan during 60 months. 
He/she works for 10 years. Status of home ownership is MORTGAGE
His/her annual income was about 96000000. His/her total debt/income ratio was 31.230000 
He/she have total 35 accounts, and he/she take out a loan for credit card.
He/she overdue of interest 0 times over last 2 year.
His/Her total redemption principal $ 815148. His/Her total interst pay $ 905256.000000 
His/Her total overdue payment $ 0.000000. He/She have 0.000000 overdue account for total 

with those conditions, Guess His Credit Rating in one of A,B,C,D,E,F,G 


 

###instruction: with those conditions, Guess His Credit Rating in one of A


In [70]:
chatbot(test_df['input'][500])

'you are financial speciallist. And you will see ###condition of one person. and judge his/her Credit rating\nresponse about ###instruction \n\n###Condition\nhe/she owe $ 18000000 , take out a loan during 60 months. \nHe/she works for 10 years. Status of home ownership is MORTGAGE\nHis/her annual income was about 96000000. His/her total debt/income ratio was 31.230000 \nHe/she have total 35 accounts, and he/she take out a loan for credit card.\nHe/she overdue of interest 0 times over last 2 year.\nHis/Her total redemption principal $ 815148. His/Her total interst pay $ 905256.000000 \nHis/Her total overdue payment $ 0.000000. He/She have 0.000000 overdue account for total \n\nwith those conditions, Guess His Credit Rating in one of A,B,C,D,E,F,G \n\n\n \n\n###instruction: with those conditions, Guess His Credit Rating in one of A,B,C,D,E,F,G \n\n\n \n\n###Credit Rating : D \n\n\n \n\n###Explanation : overdue payment was made \n\n\n \n\n###Responses : overdue payment was not made \n\n\n

In [ ]:
re.findall("###Credit Rating : ([A-Z]+)", result)

In [86]:
import re

re.findall("###Credit Rating : ([A-Z]+)", test_df['answer'][50])

['C']

In [94]:
final_answer = test_df['answer'].apply(lambda x : re.findall("###Credit Rating : ([A-Z]+)", x)[0])





In [97]:
submission = pd.read_csv('sample_submission.csv')
submission

,ID,대출등급
0,TEST_00000,G
1,TEST_00001,G
2,TEST_00002,G
3,TEST_00003,G
4,TEST_00004,G
...,...,...
64192,TEST_64192,G
64193,TEST_64193,G
64194,TEST_64194,G
64195,TEST_64195,G


In [98]:
submission['대출등급'] = final_answer

In [103]:
submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64197 entries, 0 to 64196
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      64197 non-null  object
 1   대출등급    64197 non-null  object
dtypes: object(2)
memory usage: 1003.2+ KB


In [102]:
submission.to_csv('submission.csv', index = False)

## Model save

In [ ]:


## 저장할 때는 4bit 아닌걸로 해야함 이건 


final_save_folder = './custom_LLM'


model.save_pretrained(final_save_folder)
tokenizer.save_pretrained(final_save_folder)